In [9]:
%pip install google-genai
%pip install python-dotenv
%pip install serpapi
%pip install google-search-results

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32173 sha256=bc981c00a5ef1b6252599c283babff5d72504fd0595232733799166b9b6f0caf
  Stored in directory: c:\users\varun_singhal\appdata\local\pip\cache\wheels\0c\47\f5\89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("API_KEY")

In [58]:
from serpapi import GoogleSearch

def find_flight(params):
    source, destination, travel_date, adults = params.split(",")
    params = {
        "engine": "google_flights",
        "departure_id": source,
        "arrival_id": destination,
        "outbound_date": travel_date,
        "return_date": "",
        "currency": "INR",
        "hl": "en",
        "type":2,
        "sort_by":2,
        "adults":adults,
        "stops":1,
        "api_key": "e4a5753152998398c7af7eba846dba3903fca88f928ddee14475839338d1b5f4"
        }

    search = GoogleSearch(params)
    results = search.get_dict()
    return results

In [100]:
from serpapi import GoogleSearch


def find_hotel(params):
    city, checkin_date, checkout_date, adults = params.split(",")
    params = {
    "engine": "google_hotels",
    "q": city,
    "check_in_date": checkin_date,
    "check_out_date": checkout_date,
    "adults": adults,
    "currency": "INR",
    "gl": "us",
    "hl": "en",
    "api_key": "e4a5753152998398c7af7eba846dba3903fca88f928ddee14475839338d1b5f4"
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    return results

In [101]:
function_map = {
    "find_flight":find_flight,
    "find_hotel":find_hotel
}

city_codes = {
    "delhi":"DEL",
    "bangalore":"BLR",
    "mumbai":"BOM",
    "chennai":"MAA",
    "hyderabad":"HYD",
    "kolkata":"CCU"
}

In [103]:
from datetime import datetime

current_date = str(datetime.now().date())
current_date

system_prompt = f"""You are a travel agent. By analyzing the input, you need to respond in the following format only:

<python function name>|<arguments>

Following are the list of functions available:

1. find_hotel(city, checkin, checkout, adults) : Function to find the cheapest hotel in the city for the given checkin and checkout date and number of adults.
2. find_flight(origin, destination, departure_date, adults) : Function to find the cheapest flight from the origin to the destination on the given departure date.

Consider the current date is {current_date}. Calculate the date of travel taking reference of current date if specific date is not mentioned in the query.

Lastly, consider the {city_codes} mapping and put city codes in the reponse.


"""


In [104]:
#query = "Find me a cheapest early morning flight for me and my son day after tomorrow from mumbai to delhi, which should be fast and cheapest"
query = "Find me a hotel in Bali for tomorrow"
system_prompt += query
system_prompt

"You are a travel agent. By analyzing the input, you need to respond in the following format only:\n\n<python function name>|<arguments>\n\nFollowing are the list of functions available:\n\n1. find_hotel(city, checkin, checkout, adults) : Function to find the cheapest hotel in the city for the given checkin and checkout date and number of adults.\n2. find_flight(origin, destination, departure_date, adults) : Function to find the cheapest flight from the origin to the destination on the given departure date.\n\nConsider the current date is 2025-10-08. Calculate the date of travel taking reference of current date if specific date is not mentioned in the query.\n\nLastly, consider the {'delhi': 'DEL', 'bangalore': 'BLR', 'mumbai': 'BOM', 'chennai': 'MAA', 'hyderabad': 'HYD', 'kolkata': 'CCU'} mapping and put city codes in the reponse.\n\n\nFind me a hotel in Bali for tomorrow"

In [105]:
from google import genai
client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=system_prompt
)
response = response.text
response = response.replace("\n","").split("|")
print(response)

['find_hotel', 'Bali,2025-10-09,2025-10-10,1']


In [106]:
function_response = None
if isinstance(response, list) and len(response) == 2:
    try:
        function = function_map[response[0]]
        function_response = function(response[1])
    except Exception as ex:
        print(f"Error in response: {str(ex)}")

In [108]:
result_prompt= f"""You are an travel agent, Based on the query: {query}, an external source of information has generated the following response: {function_response}. 

Analyse this response and the query and formulate the response one of the following formats only with no extra information and text:

Best Flight - DEP <Detaprture date time> - ARR <Arrival Date Time> - <Flight company / Flight number> - <Departure city> - <Arrival City> - <Flight Price>
OR
Best Hotel - <City> - <Check In Date> - <Checkout Date> - <Hotel Name> - <Price per night>

"""
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=result_prompt
)
response = response.text
response

'Best Hotel - Bali - 2025-10-09 - 2025-10-10 - NIRWANA KINTAMANI - ₹1,477\n'